In [13]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import io
from google.colab import drive
import threading
import time
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import glob
glob.glob("/content/drive/My Drive/google_notebook/M5_traindata2/*")

['/content/drive/My Drive/google_notebook/M5_traindata2/calendar2.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/calendar.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sales_train_validation.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sample_submission.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices2.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a']

In [10]:
import os.path

train_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sales_train_validation.csv")
calendar_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar.csv")
sellprices2_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices2.csv")
calendar_data2 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar2.csv")

cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "wm_yr_wk", "wday", "month", "year", "d",
        "Religious", "National", "Cultural", "Sporting", "snap", "price", "price_down", "price_up", "value"]
train_data2a0 = pd.DataFrame(index=[], columns=cols)

start_n = 0

thread_num = 15 #スレッド数

print(len(train_data))

current_num = [-1 for jj in range(thread_num)]  # 実行中のファイル番号(重複処理を防ぐ)


class traindata2Threading(threading.Thread):

    def __init__(self, thread_name):
        self.thread_name = str(thread_name)
        self.thread_number = int(thread_name)
        threading.Thread.__init__(self)

    def __str__(self):
        return self.thread_name

    def run(self):
        for ii in range(start_n, len(train_data)):
            if ((ii in current_num) == False): #その行は処理されていない
                current_num[self.thread_number] = ii
                if (os.path.isfile("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv") == False):  # ファイルがまだ作成されていない
                    train_data2a = train_data2a0.copy()
                    tra_sir = train_data.iloc[ii]
                    datas_base = tra_sir[0:6].tolist()

                    sap_mindata = sellprices2_data[sellprices2_data.item_id == datas_base[1]]  # sellpricesの中から必要なデータを抽出
                    sap_mindata = sap_mindata[sap_mindata.store_id == datas_base[4]]

                    for jj in range(1913):
                        cal_sir2 = calendar_data.iloc[jj]
                        cal2_sir2 = calendar_data2.iloc[jj]
                        sm2 = sap_mindata[sap_mindata.wm_yr_wk == cal_sir2[1]]  # sellpricesの該当行を抽出
                        try:
                            sap_sir = sm2.iloc[0]

                            datas = copy.deepcopy(datas_base)

                            kk = [cal_sir2["wm_yr_wk"], cal_sir2["wday"], cal_sir2["month"], cal_sir2["year"], cal_sir2["d"], cal2_sir2["Religious"], cal2_sir2["National"], cal2_sir2["Cultural"], cal2_sir2["Sporting"]]
                            datas = datas + kk

                            if (datas[5] == "CA"):
                                datas.append(cal2_sir2["snap_CA"])  # 自分のいる州のsnapを調査
                            elif (datas[5] == "TX"):
                                datas.append(cal2_sir2["snap_TX"])
                            elif (datas[5] == "WI"):
                                datas.append(cal2_sir2["snap_WI"])
                            else:
                                print(5 / 0)

                            kk2 = [sap_sir["sell_price"], sap_sir["price_down"], sap_sir["price_up"], tra_sir[6 + jj]]
                            datas = datas + kk2

                            kari_t = pd.Series(datas, index=cols)
                            train_data2a = train_data2a.append(kari_t, ignore_index=True)

                        except:  # 該当価格データが存在しない=販売していない
                            pass

                    train_data2a.to_csv(
                        "/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv")
                current_num[self.thread_number] = -1
                print('\r', end='')
                print(current_num, end='')


thread_list = []
for i in range(thread_num):
    thread = traindata2Threading(thread_name=i)
    thread.start()
    thread_list.append(thread)
    sleep_seconds = 1
    time.sleep(sleep_seconds)

for thread in thread_list:
    thread.join()

 642

KeyboardInterrupt: ignored

In [12]:
current_num

[643, 644, 645]